In [1]:
import scipy.io
import scipy.io
from function.sparse_learning_based import MCFS
from utility import construct_W
from utility import unsupervised_evaluation
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [35]:
ds_test.proto.value_counts()

tcp       79946
udp       63283
unas      12084
arp        2859
ospf       2595
          ...  
rdp          98
netblt       98
igmp         18
icmp         15
rtp           1
Name: proto, Length: 133, dtype: int64

In [29]:
pd.get_dummies(ds_attacks[columns], columns=string_fields)

dur  spkts  dpkts  sbytes  dbytes           rate  sttl  dttl  \
48607   0.928003     10      8     756     354      18.318906   254   252   
116905  0.000009      2      0     200       0  111111.107200   254     0   
58569   2.182624     12     10    1106     852       9.621446   254   252   
26666   0.043788    122    126    7808   15182    5640.814714    31    29   
75141   1.916999    224     36  285505    1558     135.107010   254   252   
...          ...    ...    ...     ...     ...            ...   ...   ...   
28937   0.025720     16     18    1540    1644    1283.048198    31    29   
77748   0.000005      2      0     200       0  200000.005100   254     0   
41840   1.991999     10      8     588     354       8.534141   254   252   
121899  0.508910     10     10     848    1104      37.334696    62   252   
142863  0.000004      2      0     114       0  250000.000600   254     0   

               sload         dload  ...  service_dns  service_ftp  \
48607   5.870670e+03  2.672405e+03  ...            0            0   
116905  8.888889e+07  0.000000e+00  ...            0            0   
58569   3.716627e+03  2.811295e+03  ...            0            0   
26666   1.414817e+06  2.751804e+06  ...            0            0   
75141   1.186150e+06  6.322382e+03  ...            0            0   
...              ...           ...  ...          ...          ...   
28937   4.491446e+05  4.830482e+05  ...            0            0   
77748   1.600000e+08  0.000000e+00  ...            0            0   
41840   2.128515e+03  1.244980e+03  ...            0            0   
121899  1.200998e+04  1.562555e+04  ...            0            0   
142863  1.140000e+08  0.000000e+00  ...            1            0   

        service_ftp-data  service_http  service_pop3  service_smtp  state_CON  \
48607                  0             0             0             0          0   
116905                 0             0             0             0          0   
58569                  0             0             0             0          0   
26666                  0             0             0             0          0   
75141                  0             0             0             0          0   
...                  ...           ...           ...           ...        ...   
28937                  0             0             0             0          0   
77748                  0             0             0             0          0   
41840                  0             0             0             0          0   
121899                 0             1             0             0          0   
142863                 0             0             0             0          0   

        state_FIN  state_INT  state_REQ  
48607           1          0          0  
116905          0          1          0  
58569           1          0          0  
26666           1          0          0  
75141           1          0          0  
...           ...        ...        ...  
28937           1          0          0  
77748           0          1          0  
41840           1          0          0  
121899          1          0          0  
142863          0          1          0  

[250 rows x 73 columns]

In [2]:
from sklearn import linear_model
from utility.construct_W import construct_W
def mcfs(X, n_selected_features, **kwargs):
    """
    This function implements unsupervised feature selection for multi-cluster data.

    Input
    -----
    X: {numpy array}, shape (n_samples, n_features)
        input data
    n_selected_features: {int}
        number of features to select
    kwargs: {dictionary}
        W: {sparse matrix}, shape (n_samples, n_samples)
            affinity matrix
        n_clusters: {int}
            number of clusters (default is 5)

    Output
    ------
    W: {numpy array}, shape(n_features, n_clusters)
        feature weight matrix

    Reference
    ---------
    Cai, Deng et al. "Unsupervised Feature Selection for Multi-Cluster Data." KDD 2010.
    """

    # use the default affinity matrix
    if 'W' not in kwargs:
        W = construct_W(X)
    else:
        W = kwargs['W']
    # default number of clusters is 5
    if 'n_clusters' not in kwargs:
        n_clusters = 5
    else:
        n_clusters = kwargs['n_clusters']

    # solve the generalized eigen-decomposition problem and get the top K
    # eigen-vectors with respect to the smallest eigenvalues
    W = W.toarray()
    W = (W + W.T) / 2
    W_norm = np.diag(np.sqrt(1 / W.sum(1)))
    W = np.dot(W_norm, np.dot(W, W_norm))
    WT = W.T
    W[W < WT] = WT[W < WT]
#    eigen_value, ul = scipy.linalg.eigh(a=W)  
    eigen_value, ul = scipy.sparse.linalg.eigsh(A=W, k=n_clusters, sigma=-1, which = 'LM')
#    Y = np.dot(W_norm, ul[:, -1*n_clusters-1:-1])
    Y = np.dot(W_norm, ul)
    print(ul, ul.shape, Y.shape, eigen_value)
    # solve K L1-regularized regression problem using LARs algorithm with cardinality constraint being d
    n_sample, n_feature = X.shape
    W = np.zeros((n_feature, n_clusters))
    for i in range(n_clusters):
        clf = linear_model.Lars(n_nonzero_coefs=n_selected_features)
        clf.fit(X, Y[:, i])
        W[:, i] = clf.coef_
    return W

In [19]:
!jupyter labextension list

JupyterLab v3.0.0
/home/gliber/miniconda3/share/jupyter/labextensions
        @jupyter-widgets/jupyterlab-manager v3.0.0 enabled OK (python, jupyterlab_widgets)



In [3]:
%%timeit -r1 -n1
# load data
ds_train = pd.read_csv("./data/UNSW_NB15_training-set.csv")
ds_test = pd.read_csv("./data/UNSW_NB15_testing-set.csv")
ds_attacks = pd.concat([ds_train, ds_test])
columns = list(ds_attacks.columns)
#remove proto (avoid dump)
columns = [x for x in columns if x not in ["attack_cat","id"]]
X = ds_attacks[columns]
string_fields = list(X.select_dtypes('object').columns.values)
#string_fields.remove(["label"])
X = pd.get_dummies(X, columns=string_fields)
columns = X.columns
X = X.sample(n = 20000, random_state=1)
y = np.array(X["label"])
X = np.array(X)
 
# construct affinity matrix
kwargs = {"metric": "euclidean", "neighborMode": "knn", "weightMode": "heatKernel", "k": 5, 't': 1}
#W = construct_W.construct_W(X, **kwargs)
W = construct_W(X, **kwargs)

num_fea = 100    # specify the number of selected features
num_cluster = 2    # specify the number of clusters, it is usually set as the number of classes in the ground truth

# obtain the feature weight matrix
#Weight = MCFS.mcfs(X, n_selected_features=num_fea, W=W, n_clusters=2)
Weight = mcfs(X, n_selected_features=num_fea, W=W, n_clusters=2)

# sort the feature scores in an ascending order according to the feature scores
idx = MCFS.feature_ranking(Weight)


mcfs_score = Weight.max(1)
df_ranking = pd.DataFrame({'columns':columns[idx], 'weight':mcfs_score[idx]})
df_ranking.to_csv("ranking_mcfs.csv")


# obtain the dataset on the selected features
selected_features = X[:, idx[0:num_fea]]

# perform kmeans clustering based on the selected features and repeats 20 times
nmi_total = 0
acc_total = 0
for i in range(0, 2):
    nmi, acc = unsupervised_evaluation.evaluation(X_selected=selected_features, n_clusters=num_cluster, y=y)
    nmi_total += nmi
    acc_total += acc

# output the average NMI and average ACC
print ('NMI:', float(nmi_total)/2)
print ('ACC:', float(acc_total)/2)



[[ 2.69639347e-18  2.51302763e-18]
 [-2.85920156e-18  2.16251503e-19]
 [-1.20177627e-18 -1.51006865e-19]
 ...
 [-4.41792502e-18  3.83666044e-19]
 [-7.32019070e-20  1.14564749e-18]
 [ 1.28608329e-18  1.16082186e-18]] (20000, 2) (20000, 2) [-0.78816083 -0.74043807]
NMI: 0.060503093941074365
ACC: 0.66435
8min ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [ ]:
%%timeit -r1 -n1
# load data
ds_train = pd.read_csv("./data/UNSW_NB15_training-set.csv")
ds_test = pd.read_csv("./data/UNSW_NB15_testing-set.csv")
ds_attacks = pd.concat([ds_train, ds_test])
columns = list(ds_attacks.columns)
#remove proto (avoid dump)
columns = [x for x in columns if x not in ["attack_cat","id"]]
X = ds_attacks[columns]
string_fields = list(X.select_dtypes('object').columns.values)
#string_fields.remove(["label"])
X = pd.get_dummies(X, columns=string_fields)
columns = X.columns
X = X.sample(n = 5000, random_state=1)
y = np.array(X["label"])
X = np.array(X)
 
# construct affinity matrix
kwargs = {"metric": "euclidean", "neighborMode": "knn", "weightMode": "heatKernel", "k": 5, 't': 1}
#W = construct_W.construct_W(X, **kwargs)
W = construct_W(X, **kwargs)

num_fea = 100    # specify the number of selected features
num_cluster = 2    # specify the number of clusters, it is usually set as the number of classes in the ground truth

# obtain the feature weight matrix
#Weight = MCFS.mcfs(X, n_selected_features=num_fea, W=W, n_clusters=2)
Weight = mcfs(X, n_selected_features=num_fea, W=W, n_clusters=2)

# sort the feature scores in an ascending order according to the feature scores
idx = MCFS.feature_ranking(Weight)


mcfs_score = Weight.max(1)
df_ranking = pd.DataFrame({'columns':columns[idx], 'weight':mcfs_score[idx]})
df_ranking.to_csv("ranking_mcfs.csv")


# obtain the dataset on the selected features
selected_features = X[:, idx[0:num_fea]]

# perform kmeans clustering based on the selected features and repeats 20 times
nmi_total = 0
acc_total = 0
for i in range(0, 20):
    nmi, acc = unsupervised_evaluation.evaluation(X_selected=selected_features, n_clusters=num_cluster, y=y)
    nmi_total += nmi
    acc_total += acc

# output the average NMI and average ACC
print ('NMI:', float(nmi_total)/20)
print ('ACC:', float(acc_total)/20)

In [68]:
mcfs_score = Weight.max(1)
df_ranking = pd.DataFrame({'columns':columns[idx], 'weight':mcfs_score[idx]})
df_ranking.to_csv("ranking_mcfs.csv")

In [59]:
columns[idx]

Index(['proto_ospf', 'proto_larp', 'proto_sps', 'proto_iplt',
       'proto_sprite-rpc', 'proto_udp', 'proto_ippc', 'proto_unas',
       'proto_sccopmce', 'proto_chaos',
       ...
       'proto_ptp', 'proto_prm', 'proto_pnni', 'proto_pipe', 'proto_pgm',
       'proto_nvp', 'proto_nsfnet-igp', 'proto_netblt', 'proto_narp',
       'proto_irtp'],
      dtype='object', length=197)

In [55]:
mcfs_score = Weight.max(1)
idx = np.argsort(mcfs_score, 0)
idx = idx[::-1]
df_ranking = pd.DataFrame({'columns':columns[idx], 'weight':mcfs_score})
df_ranking.to_csv("ranking_mcfc.csv")

array([2.02116972e+04, 7.63452913e+04, 5.49965449e+03, 5.44786836e+02,
       1.79575373e+00, 1.99620451e+00, 2.12807412e+00, 4.53468877e+03,
       7.62992465e-07, 6.59692019e-04, 5.94465082e+03, 1.29370225e+06,
       4.87088112e+04, 3.95663312e+04, 4.14962388e+00, 2.51745013e+03,
       1.01890367e+16, 9.15082207e-05, 4.05169255e-05, 0.00000000e+00,
       0.00000000e+00, 8.93909278e+06, 6.63267408e+04, 7.12856715e+00,
       2.20320945e+01, 4.02190798e+03, 7.09493654e+00, 1.00748468e+03,
       7.22508259e+03, 6.63673261e+02, 6.26858113e+05, 2.96344252e+02,
       4.86853349e+04, 0.00000000e+00, 3.69558015e+04, 4.74783879e+05,
       8.67482158e+02, 6.17750328e+04, 8.65911679e+17, 7.86737192e+05,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 1.73229267e+18, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 2.59820435e+18, 0.00000000e+00, 0.00000000e+00,
      

In [43]:
df_ranking = pd.DataFrame({'columns':columns[idx], 'weight':weights})
df_ranking.to_csv("ranking_autoencoder_sparce.csv")

(250, 73)